In [2]:
import numpy as np

In [3]:
def calculate_iou(box1, box2):
    new_box1 = [box1[0], box1[1], box1[0] + box1[2], box1[1] + box1[3]]
    new_box2 = [box2[0], box2[1], box2[0] + box2[2], box2[1] + box2[3]]
    
    x1 = max(new_box1[0], new_box2[0])
    y1 = max(new_box1[1], new_box2[1])
    x2 = min(new_box1[2], new_box2[2])
    y2 = min(new_box1[3], new_box2[3])

    intersection_area = max(0, x2 - x1) * max(0, y2 - y1)

    box1_area = (new_box1[2] - new_box1[0]) * (new_box1[3] - new_box1[1])
    box2_area = (new_box2[2] - new_box2[0]) * (new_box2[3] - new_box2[1])

    union_area = box1_area + box2_area - intersection_area

    if union_area == 0:
        return 0  
    iou = intersection_area / union_area
    return iou
    

In [4]:
def precision_recall(labels, predictions, iou_threshold=0.5):
    matched_labels = set()
    true_positives = 0
    
    for pred in predictions:
        for i, label in enumerate(labels):
            iou = calculate_iou(pred, label)
            if iou >= iou_threshold and i not in matched_labels:
                true_positives += 1
                matched_labels.add(i)
                break

    false_positives = len(predictions) - true_positives
    false_negatives = len(labels) - len(matched_labels)
    
    precision = true_positives / (true_positives + false_positives) if predictions else 0
    recall = true_positives / (true_positives + false_negatives) if labels else 0
    
    return precision, recall
    

In [5]:
def calculate_f1(precision, recall):
    return 2 * (precision * recall) / (precision + recall) if precision + recall else 0

In [6]:
def calculate_average_precision(precisions, recalls):
    recalls = np.concatenate([[0], recalls, [1]])
    precisions = np.concatenate([[0], precisions, [0]])

    for i in range(len(precisions) - 2, -1, -1):
        precisions[i] = max(precisions[i], precisions[i + 1])

    indices = np.where(recalls[1:] != recalls[:-1])[0]
    ap = np.sum((recalls[indices + 1] - recalls[indices]) * precisions[indices + 1])
    return ap

In [17]:
step = 1000
precisions = []
recalls = []


labels = np.array([])
predictions = np.array([])

# for i in range(1, step):
#     confident = 100/step * i     # must start with low confident first
#     predictions_i = predictions >= confident
#     precision, recall = precision_recall(labels, predictions_i)
#     precisions.append(precision)
#     recalls.append(recall)
    
# testing
recalls = [i/1000 for i in range(1, 1000)]
precisions = [(1 - (i/1000)**2)**0.5 for i in range(1, 1000)]
ap = calculate_average_precision(precisions, recalls)
print(4* ap) # calculate pi XD

    
    
    
    

3.1395554669110277


# all metric you can write:
## precision, recall, F1, AP, mAP (mean of all AP in different category)
